# Scraper tous les données et les stocker dans des listes

## les librairies à importer

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import requests

In [ ]:
import lxml.html as lh

In [ ]:
import pandas as pd

In [ ]:
from random import randint

In [ ]:
from time import sleep

In [ ]:
def fonction_url(m,a):    #  m : le mois ; a : l'année
    if m <10:
        return(f"http://www.infoclimat.fr/stations-meteo/analyses-mensuelles.php?mois=0{m}&annee={a}")
    else:
        return(f"http://www.infoclimat.fr/stations-meteo/analyses-mensuelles.php?mois={m}&annee={a}")

In [ ]:
# cette fonction suppprime tout ce qui est après "le (inclus)"
def supp_le(e):
    c = 'le'
    if c in str(e):
        e = str(e)
        return float(e[:e.index(c)])

## Le scraping des données et la creation de pandas dataframe

Cette fois on va esayer d'extraire les données de 2018

In [ ]:
Dico = {}    # this a dictionnary that stores the year numbers as keys and lst as values 
a = 2018    
lst = [] # this a list that stores all monthly data of each year 
for m in range(1,4):
        url = fonction_url(m,a)
        page = requests.get(url)# copier la page du site
        doc = lh.fromstring(page.content)# stocker tout le contenu dans doc
        tr_elements = doc.xpath('//tr')# extraire les données stockés entre <tr>..</tr> du HTML
        tr_elements = doc.xpath('//tr')

        col=[]   # une liste vide 
        i=1    # pour chaque ligne, on stocke le nom dans col
        lst_nom = ["Station","Temperature minimale extrême du mois","Moyenne du temperature minimale du mois",
                   "Temperature moyenne du mois","Moyenne des température maximale du mois","Température maximale extrême",
                   "Cumul de précipitations du mois","Heures d'ensolleiment du mois","Rafale"]
        for nom in lst_nom:
            col.append((nom,[]))   # col contient des tuples ( nom , [])
        #la première ligne est le header, les données sont stockées à partir de la deuxième ligne.
        for j in range(1,len(tr_elements)):
            T=tr_elements[j]
            #si la ligne n'est pas de taille  de T 
            if len(T)!= 9 :
                 break
            i = 0 # l'indice de la colonne
            # pour chaque element de la ligne
            for t in T.iterchildren():
                data = t.text_content() 
                # si la ligne est vide
                try:
                    data = float(data)
                except:
                    pass  
                col[i][1].append(data)  # ajouter la data à la liste vide de la ième colonne*
                i += 1
        Dict={n:colonne for (n,colonne) in col}
        df=pd.DataFrame(Dict)
        # appliquer la fonction supp_le
        df['Temperature minimale extrême du mois'] = df['Temperature minimale extrême du mois'].apply(supp_le)   
        df ['Température maximale extrême'] =  df['Température maximale extrême'].apply(supp_le)
        lst.append(df)
        sleep(randint(2,4))

convertir chaque pandas dataframe à une feuille dans un seul fichier excel qu'on le nommera données 2018

In [ ]:
# créer un excel writer en utilisant le moteur xlsxwriter
writer = pd.ExcelWriter('données2018.xlsx' , engine = 'xlsxwriter') 

# écrire chaque pandas dataframe de lst à une feuille du fichier excel

lst[0].to_excel(writer , sheet_name = 'Feuille1')
lst[1].to_excel(writer , sheet_name = 'Feuille2')
lst[2].to_excel(writer , sheet_name = 'Feuille3')
    
writer.save()

On applique la méthode ci-dessus pour scraper tous les données de 2017 à 2021, ça prend beaucoup de temps , la complexité est de O(n^4) avec n grande (d'ordre 100) 